In [9]:
#adapted from write-inputs-cluster (Ivan)
import numpy as np

In [10]:
### Read j data ###
dataj_sup = np.loadtxt('jij.sup')
dataj_imp = np.loadtxt('jij.fe.layer.rot')

dataj_sup_vec = dataj_sup[:,2:5]
dataj_imp_vec = dataj_imp[:,2:5]

unique_dist = sorted(list(set(dataj_sup[:,6])))
differences_dist = np.diff(unique_dist)

### Definitions ####
size_x = 100 # size of the spin lattice
size_y = 100
va = [0.707000,0.000000,0] ## primitive vector 1 of the lattice
vb = [0.353500,0.612300,0] ## primitive vector 2 of the lattice
pos_imp = 1 # position (site number) of the impurity on jfile.imp
max_shells = 2 # for the marriage between pristine and impurity (recommended > 1). Should be a connection
## that the pristine interaction handles
z_length = 1 # how many surfaces are there
initial_moment_vector = [0,0,1] # initial moment vector of the atoms
#######

In [16]:
# find how many decimal places there are in va and vb
maxn = 0
for i in range(len(va)):
    if int(str(va[i])[::-1].find('.')) >= maxn:
        maxn = int(str(va[i])[::-1].find('.'))
    if int(str(vb[i])[::-1].find('.')) >= maxn:
        maxn = int(str(vb[i])[::-1].find('.'))

index_imp = list(map(int, list(set(dataj_imp.transpose()[0]))))
print(index_imp)
index_imp.remove(pos_imp)
print(index_imp, maxn)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] 4


In [30]:
### print cell to inpsd.dat
print('writing cell... ', end='')
veca_cell = np.multiply(va,size_x+1)
vecb_cell = np.multiply(vb,size_y+1)
cell_file = open('cell.inpsd', 'w')
print("#### cell to inpsd.dat", file = cell_file)
print("cell" + '{0:18.10f} {1:18.10f} {2:18.10f}'.format(*veca_cell), file = cell_file)
print("    " + '{0:18.10f} {1:18.10f} {2:18.10f}'.format(*vecb_cell), file = cell_file)
print("    " + '{0:18.10f} {1:18.10f} {2:18.10f}'.format(*[0,0,z_length]), file = cell_file)
print('[ok]')

count = 0 ; thr = 1e-4
for i in range(len(differences_dist)):
    if count == max_shells:
        max_distance = unique_dist[i-1]
        break
    else:
        if differences_dist[i] > thr:
            count += 1
print(count)

writing cell... [ok]
2


In [5]:
atom = 0 ; positions = [] ; center = 0
posfile = open('posfile', 'w')
jfile = open('jfile', 'w')

### write posfile

print('writing posfile... ', end='')
mean = round(size_x/2, 0)
for i in range(size_x):
    for j in range(size_y):
        atom += 1
        p = (np.multiply(va,i)+np.multiply(vb,j))
        positions.append(p.tolist())
        print('{0:5d} {1:5d} {2:18.10f} {3:18.10f} {4:18.10f}'.format(atom, atom, *p), file = posfile)
        if i == mean and j == mean:
            center = atom
positions = np.round(positions, decimals=maxn).tolist()
print('[ok]')
print(len(positions))

writing posfile... [ok]
10000


In [6]:
interactions_center = list(np.where(dataj_imp[:,0]==pos_imp)[0])
dataj_imp_vec_center = dataj_imp[interactions_center,2:5]
#print(dataj_imp_vec_center)
sites_imp_old = list(map(int, list(dataj_imp[interactions_center,1])))
#print(sites_imp_old)
positions[center-1]
print(dataj_imp_vec_center)

[[ 1.414   0.      0.    ]
 [ 1.0605 -0.6123  0.    ]
 [ 0.707  -1.2246  0.    ]
 [ 1.0605  0.6123  0.    ]
 [ 0.707   0.      0.    ]
 [ 0.3535 -0.6123  0.    ]
 [-0.     -1.2246  0.    ]
 [ 0.707   1.2246  0.    ]
 [ 0.3535  0.6123  0.    ]
 [-0.3535 -0.6123  0.    ]
 [-0.707  -1.2246  0.    ]
 [ 0.      1.2246  0.    ]
 [-0.3535  0.6123  0.    ]
 [-0.707  -0.      0.    ]
 [-1.0605 -0.6123  0.    ]
 [-0.707   1.2246  0.    ]
 [-1.0605  0.6123  0.    ]
 [-1.414   0.      0.    ]]


In [49]:
### write "pure" impurity interactions

#print('writing jfile... ', end='')
interactions_center = list(np.where(dataj_imp[:,0]==pos_imp)[0]) 
dataj_imp_vec_center = dataj_imp[interactions_center,2:5]                    #impurities positions
sites_imp_old = list(map(int, list(dataj_imp[interactions_center,1])))       #number of the impurities
j_center_dist = dataj_imp[interactions_center,5:7]                           #interactions from the impurities

sites_imp = []
positions_imp = [] ; all_sites_imp = []

### central atom with other impurities
all_sites_imp.append(center)

for i in range(len(dataj_imp_vec_center)):
    pos_now = np.add(positions[center-1],dataj_imp_vec_center[i])
#    print(pos_now)
    pos_now = np.round(pos_now, decimals=maxn)
#    print(positions)
    ind = int(positions.index(list(pos_now)))+1
    sites_imp.append(ind)
    all_sites_imp.append(ind)
#    print(all_sites_imp)
    positions_imp.append(pos_now)
    print('{0:5d} {1:5d} {2:12.6f} {3:12.6f} {4:12.6f} {5:12.6f} {6:12.6f}'.format(center, ind, *dataj_imp_vec_center[i], *j_center_dist[i]), file = jfile)

all_sites_imp = list(set(all_sites_imp))
print(len(all_sites_imp))

19


In [51]:
### write a *.xyz file with 'Co' format as the impurities and 'Nb' as the pristine
print('writing system.xyz.. ', end='')
system_file = open('system.xyz', 'w')

print(str(len(positions)) + '\n', file = system_file)
for i in range(len(positions)):
    atom_now = (i + 1)
    pos_now = positions[atom_now-1]
    pos_now = np.round(pos_now, decimals=maxn)
    if not atom_now in all_sites_imp:
        print("Nb " + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*pos_now), file = system_file)
    else:
        print("Co " + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*pos_now), file = system_file)
print('[ok]')


writing system.xyz.. [ok]
